# Práctica 5. Detección de características

#### EJERCICIO 1
#### Apartado A

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tkinter as tk
from tkinter import filedialog, messagebox

# Función para aplicar SIFT y detectar características
#img, nfeatures=0, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6
def sift_detection(img, nfeatures, contrastThreshold, edgeThreshold, sigma):
    sift = cv2.SIFT_create(nfeatures=nfeatures, contrastThreshold=contrastThreshold,
                           edgeThreshold=edgeThreshold, sigma=sigma)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    
    img_with_keypoints = cv2.drawKeypoints(img, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    return img_with_keypoints

# Función para cargar la imagen
def load_image():
    global img
    file_path = filedialog.askopenfilename()
    if file_path:
        img = cv2.imread(file_path)
        cv2.imshow("Imagen cargada", img)
        cv2.waitKey(0)
        show_main_menu()

# Función para mostrar el menú principal
def show_main_menu():
    clear_buttons()
    # Crear sliders para los parámetros de SIFT
    global nfeatures_slider, contrast_slider, edge_slider, sigma_slider
    
    tk.Label(root, text="Número de características (nfeatures):").pack()
    nfeatures_slider = tk.Scale(root, from_=0, to=1000, orient=tk.HORIZONTAL)
    nfeatures_slider.pack()

    tk.Label(root, text="Umbral de contraste (contrastThreshold):").pack()
    contrast_slider = tk.Scale(root, from_=0, to=1, resolution=0.01, orient=tk.HORIZONTAL)
    contrast_slider.pack()

    tk.Label(root, text="Umbral de borde (edgeThreshold):").pack()
    edge_slider = tk.Scale(root, from_=0, to=100, orient=tk.HORIZONTAL)
    edge_slider.pack()

    tk.Label(root, text="Sigma:").pack()
    sigma_slider = tk.Scale(root, from_=0.1, to=5.0, resolution=0.1, orient=tk.HORIZONTAL)
    sigma_slider.pack()

    sift_btn = tk.Button(root, text="Detectar SIFT", command=detect_sift)
    sift_btn.pack(pady=10)

# Función para ejecutar SIFT
def detect_sift():
    if 'img' in globals():
        nfeatures = nfeatures_slider.get()
        contrastThreshold = contrast_slider.get()
        edgeThreshold = edge_slider.get()
        sigma = sigma_slider.get()
        
        img_sift = sift_detection(img, nfeatures=nfeatures, contrastThreshold=contrastThreshold,
                                  edgeThreshold=edgeThreshold, sigma=sigma)
        cv2.imshow("Características detectadas", img_sift)
        cv2.waitKey(0)
        show_main_menu()
    else:
        messagebox.showerror("Error", "Cargue una imagen primero.")

# Función para limpiar los botones
def clear_buttons():
    for widget in root.winfo_children():
        widget.destroy()

# Crear la interfaz gráfica
root = tk.Tk()
root.title("Procesador de Imágenes")

# Botón para cargar la imagen
load_btn = tk.Button(root, text="Cargar Imagen", command=load_image)
load_btn.pack(pady=10)

# Ejecutar la interfaz
root.mainloop()
cv2.destroyAllWindows()


#### Apartado B, C y D

In [4]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox

# Variables globales
img = None
area_img = None
roi = None
transformed_images = []

# Función para aplicar el efecto de barril (distorsión)
def barrel_distortion(img, k):
    h, w = img.shape[:2]
    x, y = np.meshgrid(np.arange(w), np.arange(h))
    x = (x - w / 2) / (w / 2)
    y = (y - h / 2) / (h / 2)
    r = np.sqrt(x ** 2 + y ** 2)
    theta = 1 + k * r ** 2
    map_x = (x * theta + 1) * w / 2
    map_y = (y * theta + 1) * h / 2
    distorted_img = cv2.remap(img, map_x.astype(np.float32), map_y.astype(np.float32), interpolation=cv2.INTER_LINEAR)
    return distorted_img

# Función para aplicar transformaciones a la imagen
def transform_image(img, angle=0, scale=1.0, translation=(0, 0), barrel_k=0.0):
    rows, cols = img.shape[:2]
    # Matriz de rotación
    M_rot = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, scale)
    # Aplicar traslación
    M_rot[0, 2] += translation[0]
    M_rot[1, 2] += translation[1]
    transformed_img = cv2.warpAffine(img, M_rot, (cols, rows))
    # Aplicar efecto de barril
    transformed_img = barrel_distortion(transformed_img, barrel_k)
    return transformed_img

# Función para cargar la imagen
def load_image():
    global img
    file_path = filedialog.askopenfilename()
    if file_path:
        img = cv2.imread(file_path)
        cv2.imshow("Imagen cargada", img)
        create_variants_menu()

# Función para mostrar el menú de transformación
def create_variants_menu():
    clear_buttons()
    
    # Etiqueta de Transformaciones
    label_transformations = tk.Label(root, text="Transformaciones", font=("Arial", 12, "bold"))
    label_transformations.pack(pady=5)

    global angle_slider, scale_slider, translation_x_slider, translation_y_slider, barrel_slider
    angle_slider = tk.Scale(root, from_=-180, to=180, label="Ángulo de Rotación", orient=tk.HORIZONTAL)
    angle_slider.pack(pady=5)
    
    scale_slider = tk.Scale(root, from_=0.1, to=3.0, resolution=0.1, label="Escala", orient=tk.HORIZONTAL)
    scale_slider.set(1.0)
    scale_slider.pack(pady=5)
    
    translation_x_slider = tk.Scale(root, from_=-200, to=200, resolution=1, label="Traslación X", orient=tk.HORIZONTAL)
    translation_x_slider.pack(pady=5)
    
    translation_y_slider = tk.Scale(root, from_=-200, to=200, resolution=1, label="Traslación Y", orient=tk.HORIZONTAL)
    translation_y_slider.pack(pady=5)

    # Trackbar para efecto de barril
    barrel_slider = tk.Scale(root, from_=-0.5, to=0.5, resolution=0.01, label="Intensidad de Efecto Barril", orient=tk.HORIZONTAL)
    barrel_slider.pack(pady=5)

    # Botón para aplicar transformaciones
    apply_transformation_btn = tk.Button(root, text="Aplicar Transformación", command=apply_transformation)
    apply_transformation_btn.pack(pady=5)
    
    # Botón para seleccionar área de interés
    select_area_btn = tk.Button(root, text="Seleccionar Área de Interés", command=select_area_ui)
    select_area_btn.pack(pady=5)
    
    # Botón para buscar el área de interés
    search_area_btn = tk.Button(root, text="Buscar Área de Interés", command=search_area)
    search_area_btn.pack(pady=5)

# Función para aplicar la transformación y mostrar la imagen transformada
def apply_transformation():
    if img is not None:
        angle = angle_slider.get()
        scale = scale_slider.get()
        translation = (translation_x_slider.get(), translation_y_slider.get())
        barrel_k = barrel_slider.get()  # Obtener el valor de la distorsión de barril
        transformed_img = transform_image(img, angle, scale, translation, barrel_k)

        cv2.imshow("Imagen Transformada", transformed_img)

        # Guardar la imagen transformada
        transformed_images.append(transformed_img)
    else:
        messagebox.showerror("Error", "Cargue una imagen primero.")

# Función para seleccionar un área de interés
def select_area(img):
    roi = cv2.selectROI("Seleccione el área de interés", img, fromCenter=False, showCrosshair=True)
    if roi[2] > 0 and roi[3] > 0:  # Verificar si el ROI es válido
        area_img = img[int(roi[1]):int(roi[1] + roi[3]), int(roi[0]):int(roi[0] + roi[2])]
        return area_img, roi
    else:
        messagebox.showerror("Error", "Área de interés no seleccionada.")
        return None, None

# Función mejorada para buscar el área de interés en la imagen transformada
def find_area_in_transformed_image(transformed_img, template_img, roi):
    # Verificar si el ROI está dentro de los límites de la imagen transformada
    roi_x, roi_y, roi_w, roi_h = roi
    if roi_x < 0 or roi_y < 0 or roi_x + roi_w > transformed_img.shape[1] or roi_y + roi_h > transformed_img.shape[0]:
        # Mostrar mensaje de error si el ROI está fuera de los límites
        messagebox.showerror("Error", "El área seleccionada está fuera de la imagen transformada. No se buscarán características aleatorias.")
        return None
    
    # Convertir a escala de grises
    gray_transformed = cv2.cvtColor(transformed_img, cv2.COLOR_BGR2GRAY)
    gray_template = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)
    
    # Crear el detector SIFT y calcular keypoints y descriptores
    sift = cv2.SIFT_create()
    kp_transformed, des_transformed = sift.detectAndCompute(gray_transformed, None)
    kp_template, des_template = sift.detectAndCompute(gray_template, None)
    
    # Verificar que haya descriptores válidos
    if des_transformed is None or des_template is None:
        messagebox.showerror("Error", "No se encontraron suficientes puntos de interés en las imágenes.")
        return None
    
    # Configurar el matcher usando FLANN para mayor robustez
    index_params = dict(algorithm=1, trees=5)  # Algoritmo y árboles de búsqueda para FLANN
    search_params = dict(checks=50)  # Número de comprobaciones para cada punto
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    
    # Aplicar la coincidencia con FLANN y filtrar coincidencias con el filtro de Lowe (ratio test)
    matches = flann.knnMatch(des_template, des_transformed, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:  # Filtro de Lowe para coincidencias válidas
            good_matches.append(m)
    
    # Verificar si se encontraron coincidencias suficientes
    if len(good_matches) < 10:  # Ajusta este valor según sea necesario
        messagebox.showerror("Error", "No se encontraron coincidencias válidas en el área de interés transformada.")
        return None
    
    # Dibujar las coincidencias en la imagen transformada
    img_matches = cv2.drawMatches(template_img, kp_template, transformed_img, kp_transformed, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    return img_matches



# Función para buscar el área de interés en la imagen transformada seleccionada
def search_area():
    if len(transformed_images) > 0 and area_img is not None:
        transformed_img = transformed_images[-1]

        # Verificar si el ROI está dentro de los límites de la imagen transformada
        roi_x, roi_y, roi_w, roi_h = roi
        if roi_x < 0 or roi_y < 0 or roi_x + roi_w > transformed_img.shape[1] or roi_y + roi_h > transformed_img.shape[0]:
            messagebox.showerror("Error", "No se pudo encontrar puntos de interés en el área seleccionada porque se encuentra fuera de la imagen.")
            return

        match_img = find_area_in_transformed_image(transformed_img, area_img, roi)
        cv2.imshow("Área Encontrada en Imagen Transformada", match_img)
    else:
        messagebox.showerror("Error", "Seleccione una imagen y un área de interés primero.")

# Función para limpiar los botones
def clear_buttons():
    for widget in root.winfo_children():
        widget.destroy()

# Función para seleccionar área
def select_area_ui():
    global area_img, roi
    if img is not None:
        area_img, roi = select_area(img)
        if area_img is None:
            messagebox.showerror("Error", "No se pudo seleccionar el área.")
    else:
        messagebox.showerror("Error", "Cargue una imagen primero.")

# Crear la interfaz gráfica
root = tk.Tk()
root.title("Procesador de Imágenes")

# Botón para cargar la imagen
load_btn = tk.Button(root, text="Cargar Imagen", command=load_image)
load_btn.pack(pady=10)

# Ejecutar la interfaz
root.mainloop()
cv2.destroyAllWindows()
